In [1]:
# Styling notebook
from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

In [43]:
from Scanner import Scanner

class Parser :
    def __init__(self, source) :
        self.source = source
        self.scanner = Scanner(source)
        self.start = self.Stmts
        self.currentToken = None
        self.currentValue = None
        self.IR = []
    
    def error(self,message) :
        print("Error: " + message)
       
    def emit(self,tag,item) :
        self.IR.append([tag,item])
        
    def scan(self) :
        self.currentToken, self.currentValue = self.scanner.nextToken()
        
    def expect(self,token) :
        if self.currentToken != token :
            self.error(token + ' expected')
        else :
            self.scan()
    
    def parse(self) :
        self.scanner.init()
        self.scan()
        self.start()
        return self.IR
    
    def Stmts(self) :
        self.Stmt()
        while (self.currentToken == 'semicolon') :
            self.scan()
            self.Stmt()
    
    def Stmt(self) :
        self.Assign()
    
    def Assign(self) :
        self.LVal()
        self.expect('assignSym')
        self.AExpr()
        self.emit('SOp','STORE')
        
    def LVal(self) :
        if self.currentToken == 'identifier' :
            self.emit('LValue',self.currentValue)
            self.scan()

    def AExpr(self) :
        self.Term()
        while self.currentToken in {'plusSym', 'minusSym',} :
            op = self.currentToken
            self.scan()
            self.Term()
            self.emit('AOp',op)

    def Term(self) :
        self.Factor()
        while self.currentToken in {'timesSym', 'divSym',} :
            op = self.currentToken
            self.scan()
            self.Factor()
            self.emit('AOp',op)
            
    def Factor(self) :
        if self.currentToken == 'lParen' :
            self.scan()
            self.AExpr()
            self.expect('rParen');
                
        if self.currentToken == 'identifier' :
            self.emit('RValue',self.currentValue)
            self.scan()
        
        if self.currentToken == 'numConstant' :
            self.emit('NumC',self.currentValue)
            self.scan() 

In [45]:
s = "xyz = xyz + 5*(a + 4) - 6"
p = Parser(s)
p.parse()

[['LValue', 'xyz'],
 ['RValue', 'xyz'],
 ['NumC', 5],
 ['RValue', 'a'],
 ['NumC', 4],
 ['AOp', 'plusSym'],
 ['AOp', 'timesSym'],
 ['AOp', 'plusSym'],
 ['NumC', 6],
 ['AOp', 'minusSym'],
 ['SOp', 'STORE']]